In [1]:
from custom_models import *
from inception_utils import *
from image_processing import *
from video_processing import *
import pickle
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import time

Using TensorFlow backend.


In [2]:
def get_predictions(model, model_flow, num_labels, classes_dictionary,  w, fnb):    
    start = time.time()

    root_path1 = 'full_videos_dataset/frames/'
    root_path2 = 'full_videos_dataset/flow_images/'

    print('\n\nCurrently getting predictions with weighted average ( rgb =',w[0],', flow =',w[1],' )')

    conf_mat = np.zeros((num_labels,num_labels))
    top5_correct = 0
    vids_per_class = np.zeros(num_labels) #keep it for later stats extraction

    pred_out = open('results/video_sequence_results/combo'+str(fnb)+'/video_sequence_predictions_'+str(w[0])+'_'+str(w[1])+'.txt','w')

    cntr,N=0,3783
    for line in open('data_a/ucf101_split1_testVideos.txt','r'):       
        videoname1 = root_path1 + line.split()[0].split('/')[1]
        videoname2 = root_path2 + line.split()[0].split('/')[1]
        label = int(line.split()[1])

        vids_per_class[label] +=1

        #get mean of every column (101 of them) to find later greatest value among 101 and get predicted class
        if(w[0]==0.0):
            rgb_pred = np.zeros((101))
            flow_pred = get_video_pred(videoname2,model_flow,'flow_images')
        elif(w[1]==0.0):
            rgb_pred = get_video_pred(videoname1,model,'frames')
            flow_pred = np.zeros((101))
        else:
            rgb_pred = get_video_pred(videoname1,model,'frames')
            flow_pred = get_video_pred(videoname2,model_flow,'flow_images')

        weighted_pred = w[0]*rgb_pred + w[1]*flow_pred

        conf_mat[weighted_pred.argmax(),label] += 1  

        pred_out.write('{:30} was classified as : {:25}'.format(line.split()[0].split('/')[1],classes_dictionary[weighted_pred.argmax()]))#,'with max_pred={:.4f}'.format(weighted_pred.max()))
        pred_out.write('\n\nTop 5 predictions\n-----------------\n')

        ind = np.argpartition(weighted_pred, -5)[-5:]
        for i,cnt in zip(reversed(ind[np.argsort(weighted_pred[ind])]),range(5)):
            pred_out.write(str(cnt+1)+' '+classes_dictionary[i]+'\n')#, weighted_pred[i])
            if(i==label):
                top5_correct += 1
        pred_out.write('\n'+'-'*73+'\n')

        cntr+=1
        print('{:.2f}'.format(cntr/N*100), end='',flush=True)
        print('\r', end='')
        del rgb_pred,flow_pred,weighted_pred
    pred_out.close()

    #######################
    accuracy = np.trace(conf_mat) / np.sum(conf_mat)
    print('\tAccuracy is {:.3f}'.format(accuracy*100),'%')

    top5_accuracy = top5_correct / np.sum(conf_mat)
    print('\tTop-5 Accuracy is {:.3f}'.format(top5_accuracy*100),'%')

    #######################
    df_cm = pd.DataFrame(conf_mat, index = [classes_dictionary[i] for i in ([i for i in range(num_labels)])],
                                 columns = [classes_dictionary[i] for i in ([i for i in range(num_labels)])])
    sn.set(rc={'figure.figsize':(50,50)})
    sn.heatmap(df_cm, annot=True).figure.savefig('results/video_sequence_results/combo'+str(fnb)+'/video_sequence_conf_mat_'+str(w[0])+'_'+str(w[1])+'.png', dpi=100)
    plt.close() 
    
    sn.set(rc={'figure.figsize':(30,30)})    
    sn.heatmap(df_cm, annot=False).figure.savefig('results/video_sequence_results/combo'+str(fnb)+'/video_sequence_conf_mat_no_annot_'+str(w[0])+'_'+str(w[1])+'.png', dpi=100)
    plt.close() 
    
    np.save('results/video_sequence_results/combo'+str(fnb)+'/conf_mat_'+str(w[0])+'_'+str(w[1])+'.npy',conf_mat)
    
    #######################
    stats_out = open('results/video_sequence_results/combo'+str(fnb)+'/video_sequence_stats_'+str(w[0])+'_'+str(w[1])+'.txt','w')    
    for i in range(num_labels):
        stats_out.write('Classified correctly '+str(int(conf_mat[i,i]))+' of '+str(int(vids_per_class[i]))+' for class '+classes_dictionary[i]+'\n')
    stats_out.close()

    print('\tExecution time {:.2f}'.format(time.time()-start))

In [3]:
def get_predictions_combo(model, model_flow, num_labels, classes_dictionary, weights, fnb):
    start = time.time()

    get_predictions(model, model_flow, num_labels, classes_dictionary,  weights[0], fnb)    
    get_predictions(model, model_flow, num_labels, classes_dictionary,  weights[1], fnb)
    get_predictions(model, model_flow, num_labels, classes_dictionary,  weights[2], fnb)    
    get_predictions(model, model_flow, num_labels, classes_dictionary,  weights[3], fnb)
    get_predictions(model, model_flow, num_labels, classes_dictionary,  weights[4], fnb)
    
    print('\n\nExecution time {:.2f}'.format(time.time()-start))

In [4]:
seq_input = (seq_len,crop_height,crop_width,3)

classes_dictionary = pickle.load(open('data_a/action_dictionary.p','rb'))

rgb_path = 'saved_models/lstm_rgb.hdf5'
flow_paths = ['saved_models/lstm_flow.hdf5','saved_models/lstm_flow_1024.hdf5']

model = LSTMCaffeDonahueFunctional(seq_input=seq_input, num_labels=num_labels)
model.load_weights(rgb_path)

weights0 = [1.0, 0.0]
weights1 = [0.67, 0.33]
weights2 = [0.5, 0.5]
weights3 = [0.33, 0.67]
weights4 = [0.0, 1.0]

weights={}
weights[0]=weights0
weights[1]=weights1
weights[2]=weights2
weights[3]=weights3
weights[4]=weights4

In [5]:
# LSTM_RGB + LSTM_FLOW_512
model_flow = LSTMCaffeDonahueFunctional(seq_input=seq_input,num_labels=num_labels)
model_flow.load_weights(flow_paths[0])
get_predictions_combo(model, model_flow, num_labels, classes_dictionary,  weights, 0)



Currently getting predictions with weighted average ( rgb = 1.0 , flow = 0.0  )
	Accuracy is 56.490 %
	Top-5 Accuracy is 79.672 %


Currently getting predictions with weighted average ( rgb = 0.67 , flow = 0.33  )
	Accuracy is 66.323 %
	Top-5 Accuracy is 92.598 %


Currently getting predictions with weighted average ( rgb = 0.5 , flow = 0.5  )
	Accuracy is 74.623 %
	Top-5 Accuracy is 93.233 %


Currently getting predictions with weighted average ( rgb = 0.67 , flow = 0.33  )
	Accuracy is 77.372 %
	Top-5 Accuracy is 93.312 %


Currently getting predictions with weighted average ( rgb = 0.0 , flow = 1.0  )
	Accuracy is 73.962 %
	Top-5 Accuracy is 91.885 %


In [6]:
# LSTM_RGB + LSTM_FLOW_1024
model_flow = LSTMCaffeDonahueFunctional(seq_input=seq_input,num_labels=num_labels,check1024=True)
model_flow.load_weights(flow_paths[1])
get_predictions_combo(model, model_flow, num_labels, classes_dictionary,  weights, 1)   



Currently getting predictions with weighted average ( rgb = 1.0 , flow = 0.0  )
	Accuracy is 56.490 %
	Top-5 Accuracy is 79.672 %


Currently getting predictions with weighted average ( rgb = 0.67 , flow = 0.33  )
	Accuracy is 66.852 %
	Top-5 Accuracy is 91.673 %


Currently getting predictions with weighted average ( rgb = 0.5 , flow = 0.5  )
	Accuracy is 74.280 %
	Top-5 Accuracy is 92.070 %


Currently getting predictions with weighted average ( rgb = 0.67 , flow = 0.33  )
	Accuracy is 76.897 %
	Top-5 Accuracy is 91.938 %


Currently getting predictions with weighted average ( rgb = 0.0 , flow = 1.0  )
	Accuracy is 73.619 %
	Top-5 Accuracy is 90.114 %
